**Install Required Packages**


In [ ]:
# Install required packages
!pip install pdfminer.six
!pip install streamlit
!pip install pickle5
!pip install langchain
!pip install langchain-groq
!pip install faiss-cpu
!pip install huggingface_hub
!pip install -U langchain-community


**Import Libraries and Initialization**

In [ ]:
from pdfminer.high_level import extract_text
import os
import pickle
import time
from langchain_groq import ChatGroq
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA


**Initialize LLM (Language Model)**

In [ ]:
language_model = ChatGroq(temperature=0, 
                          groq_api_key="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", 
                          model_name="llama-3.1-70b-versatile")


**File Upload**

In [ ]:
from google.colab import files
uploaded_docs = files.upload()


**Extract and Split Text from Uploaded PDFs**

In [ ]:
def extract_and_split_text_from_docs():
    entire_text = ""

    # Extract text from all uploaded PDFs
    for uploaded_doc in uploaded_docs.keys():
        doc_text = extract_text(uploaded_doc)
        entire_text += doc_text + "\n"

    # Split text into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    text_chunks = text_splitter.split_text(entire_text)

    return text_chunks


**Build FAISS Index from Text Chunks**

In [ ]:
def build_faiss_index_from_chunks(text_chunks):
    # Create embeddings and vector store
    embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    faiss_vector_store = FAISS.from_texts(text_chunks, embeddings_model)

    # Save FAISS index
    print("Building FAISS index...")
    time.sleep(2)

    # Save the FAISS index to a pickle file
    index_file_path = 'faiss_store_groq.pkl'
    with open(index_file_path, "wb") as f:
        pickle.dump(faiss_vector_store, f)

    print("Text extracted and FAISS index saved.")


**Query Input and Retrieval**

In [ ]:
user_query = input("Ask a Question: ")
if user_query:
    index_file_path = 'faiss_store_groq.pkl'
    if os.path.exists(index_file_path):
        with open(index_file_path, "rb") as f:
            faiss_vector_store = pickle.load(f)
            retrieval_chain = RetrievalQA.from_llm(llm=language_model, retriever=faiss_vector_store.as_retriever())

        # Get response
        query_result = retrieval_chain.run(user_query)

        # Display answer
        print("Answer:")
        print(query_result)
